# Wortbestandteile finden und reguläre Ausdrücke (regex) in Python und Pandas

In [2]:
import numpy as np
import pandas as pd
import re

from rx.linq.observable.expand import expand

# Reguläre Ausdrücke - Regular Expressions

Reguläre Ausdrücke sind das fehlende Puzzleteil, um Muster in Strings zu erkennen. Sie verwenden dafür eine Kombination aus Spezialbuchstaben, die festlegt, auf welche flexiblen Zeichenkombinationen Strings durchsucht werden sollen (das sogenannte "regex pattern").

In Python kriegen wir Zugang zu regulären Ausdrücken über das Modul `re` und auch über `pandas`.

Regex lernen: [Link](https://regexone.com/lesson/introduction_abcs)

# Regex in Pandas

In Pandas DataFrames haben wir in Text-Spalten Zugang zu Text-bearbeitenden Funktionen unter `DataFrame["Spalte"].str.`.
Dabei werden regex-Funktionalitäten immer auf die gesamte Spalte angewandt (analog zu .dt. bei datetime64[ns]-Spalten).

Diese Funktionen greifen in der Regel ebenfalls auf re package zurück, wodurch wir regex für die Suche von Strings nutzen können.

In [3]:
titanic_df = pd.read_csv('titanic.csv',
                         usecols=['Survived', 'Name', 'Sex', 'Ticket'])
titanic_df.head()

,Survived,Name,Sex,Ticket
0,0,"Braund, Mr. Owen Harris",male,A/5 21171
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599
2,1,"Heikkinen, Miss. Laina",female,STON/O2. 3101282
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803
4,0,"Allen, Mr. William Henry",male,373450


In [4]:
# Alle Namen, die auf 'An' anfangen (als Boolean Mask):
titanic_df['Name'].str.startswith('An')

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888    False
889    False
890    False
Name: Name, Length: 891, dtype: bool

In [5]:
# Der entsprechende Ausschnitt aus den Daten:
titanic_df[titanic_df['Name'].str.startswith('An')]

,Survived,Name,Sex,Ticket
13,0,"Andersson, Mr. Anders Johan",male,347082
68,1,"Andersson, Miss. Erna Alexandra",female,3101281
91,0,"Andreasson, Mr. Paul Edvin",male,347466
119,0,"Andersson, Miss. Ellis Anna Maria",female,347082
144,0,"Andrew, Mr. Edgardo Samuel",male,231945
146,1,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,350043
192,1,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,350046
275,1,"Andrews, Miss. Kornelia Theodosia",female,13502
460,1,"Anderson, Mr. Harry",male,19952
518,1,"Angle, Mrs. William A (Florence ""Mary"" Agnes H...",female,226875


In [6]:
# Alle Namen, die auf 'er' enden:
titanic_df[titanic_df['Name'].str.endswith('er')]

,Survived,Name,Sex,Ticket
27,0,"Fortune, Mr. Charles Alexander",male,19950
51,0,"Nosworthy, Mr. Richard Cater",male,A/4. 39886
57,0,"Novel, Mr. Mansouer",male,2697
92,0,"Chaffee, Mr. Herbert Fuller",male,W.E.P. 5734
153,0,"van Billiard, Mr. Austin Blyler",male,A/5. 851
219,0,"Harris, Mr. Walter",male,W/C 14208
270,0,"Cairns, Mr. Alexander",male,113798
340,1,"Navratil, Master. Edmond Roger",male,230080
355,0,"Vanden Steen, Mr. Leo Peter",male,345783
357,0,"Funk, Miss. Annie Clemmer",female,237671


In [7]:
# Alle Namen, die den String 'Dr.' enthalten (erstmal die Wahrheitsmaske):
titanic_df['Name'].str.contains('Dr. ')

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888    False
889    False
890    False
Name: Name, Length: 891, dtype: bool

In [8]:
# Hier machen wir uns zunutze, dass nach Dr. immer ein Leerzeichen kommen muss:
titanic_df[titanic_df['Name'].str.contains('Dr. ')]

,Survived,Name,Sex,Ticket
245,0,"Minahan, Dr. William Edward",male,19928
317,0,"Moraweck, Dr. Ernest",male,29011
398,0,"Pain, Dr. Alfred",male,244278
632,1,"Stahelin-Maeglin, Dr. Max",male,13214
660,1,"Frauenthal, Dr. Henry William",male,PC 17611
766,0,"Brewe, Dr. Arthur Jackson",male,112379
796,1,"Leader, Dr. Alice (Farnham)",female,17465


In [9]:
# Alternative Lösung mit einfachen regulärem Ausdruck.
# Die Sonderbedeutung von '.' wird mit '\' ausgehebelt:
titanic_df[titanic_df['Name'].str.contains(r'Dr\.', regex=True)]

,Survived,Name,Sex,Ticket
245,0,"Minahan, Dr. William Edward",male,19928
317,0,"Moraweck, Dr. Ernest",male,29011
398,0,"Pain, Dr. Alfred",male,244278
632,1,"Stahelin-Maeglin, Dr. Max",male,13214
660,1,"Frauenthal, Dr. Henry William",male,PC 17611
766,0,"Brewe, Dr. Arthur Jackson",male,112379
796,1,"Leader, Dr. Alice (Farnham)",female,17465


In [10]:
# Nur Doktoren mit doppeltem Vornamen (mit regulärem Ausdruck): 
titanic_df[titanic_df['Name'].str.contains(r'Dr\.\s[a-z]+\s[a-z]+', regex=True, flags=re.I)]

,Survived,Name,Sex,Ticket
245,0,"Minahan, Dr. William Edward",male,19928
660,1,"Frauenthal, Dr. Henry William",male,PC 17611
766,0,"Brewe, Dr. Arthur Jackson",male,112379


In [11]:
# Aufgabe 1:
# Gesucht sind Passagiere, die ein Mr. im Namen enthalten und auf den Buchstaben E anfangen!
mr_E = r'Mr\.\sE'
titanic_df[titanic_df['Name'].str.contains(mr_E, flags=re.I)]

,Survived,Name,Sex,Ticket
33,0,"Wheadon, Mr. Edward H",male,C.A. 24579
34,0,"Meyer, Mr. Edgar Joseph",male,PC 17604
37,0,"Cann, Mr. Ernest Charles",male,A./5. 2152
54,0,"Ostby, Mr. Engelhart Cornelius",male,113509
67,0,"Crease, Mr. Ernest James",male,S.P. 3464
90,0,"Christmann, Mr. Emil",male,343276
115,0,"Pekoniemi, Mr. Edvard",male,STON/O 2. 3101294
135,0,"Richard, Mr. Emile",male,SC/PARIS 2133
144,0,"Andrew, Mr. Edgardo Samuel",male,231945
232,0,"Sjostedt, Mr. Ernst Adolf",male,237442


In [12]:
# Aufgabe 2:
# Nur verheiratete Frauen und deren (erster) Vorname fünf alphabetische Zeichen lang ist:
mrs = r'Mrs\.\s\w{5}\b'
titanic_df[titanic_df['Name'].str.contains(mrs, flags=re.I)]

,Survived,Name,Sex,Ticket
8,1,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742
40,0,"Ahlin, Mrs. Johan (Johanna Persdotter Larsson)",female,7546
49,0,"Arnold-Franchi, Mrs. Josef (Josefine Franchi)",female,349237
52,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,PC 17572
142,1,"Hakkarainen, Mrs. Pekka Pietari (Elin Matilda ...",female,STON/O2. 3101279
161,1,"Watt, Mrs. James (Elizabeth ""Bessie"" Inglis Mi...",female,C.A. 33595
194,1,"Brown, Mrs. James Joseph (Margaret Tobin)",female,PC 17610
230,1,"Harris, Mrs. Henry Birkhardt (Irene Wallach)",female,36973
299,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,PC 17558
316,1,"Kantor, Mrs. Sinai (Miriam Sternin)",female,244367


In [13]:
# Split: Wenn man mal wieder etwas teilen will.
# Hier teilen wir Nachnamen und den Teil mit Vornamen nach dem Komma.
# Das Zeichen, an dem aufgetrennt wird ist das Komma:
titanic_df['Name'].str.split(', ')

0                              [Braund, Mr. Owen Harris]
1      [Cumings, Mrs. John Bradley (Florence Briggs T...
2                               [Heikkinen, Miss. Laina]
3         [Futrelle, Mrs. Jacques Heath (Lily May Peel)]
4                             [Allen, Mr. William Henry]
                             ...                        
886                              [Montvila, Rev. Juozas]
887                       [Graham, Miss. Margaret Edith]
888           [Johnston, Miss. Catherine Helen "Carrie"]
889                              [Behr, Mr. Karl Howell]
890                                [Dooley, Mr. Patrick]
Name: Name, Length: 891, dtype: object

In [14]:
# Wäre es nicht cool, das Ganze direkt auf zwei Spalten aufteilen zu können?
# Geht auch! Mit expand:
titanic_df['Name'].str.split(', ', expand=True)

,0,1
0,Braund,Mr. Owen Harris
1,Cumings,Mrs. John Bradley (Florence Briggs Thayer)
2,Heikkinen,Miss. Laina
3,Futrelle,Mrs. Jacques Heath (Lily May Peel)
4,Allen,Mr. William Henry
...,...,...
886,Montvila,Rev. Juozas
887,Graham,Miss. Margaret Edith
888,Johnston,"Miss. Catherine Helen ""Carrie"""
889,Behr,Mr. Karl Howell


In [15]:
# Jetzt wäre es noch ganz cool, Vor- und Nachnamen direkt zum Datensatz in einem Zug hinzuzufügen:
# Auch das geht:
titanic_df[['Last_Name', 'First_Name']] = titanic_df['Name'].str.split(', ', expand=True)

In [16]:
titanic_df.head()

,Survived,Name,Sex,Ticket,Last_Name,First_Name
0,0,"Braund, Mr. Owen Harris",male,A/5 21171,Braund,Mr. Owen Harris
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,Cumings,Mrs. John Bradley (Florence Briggs Thayer)
2,1,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,Heikkinen,Miss. Laina
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,Futrelle,Mrs. Jacques Heath (Lily May Peel)
4,0,"Allen, Mr. William Henry",male,373450,Allen,Mr. William Henry


In [17]:
# Ganz klare (allgemeine) Vorstellungen .match():
# Der Nachname beginnt mit B und ist acht Zeichen lang,
# gefolgt von Mr., gefolgt von einem sieben Zeichen langen ersten Vornamen 
# und einem vier Zeichen langen zweiten Vornamen.
# Erst das Pattern + Wahrheitsmaske:
pattern = r'^b[a-z]{7}, Mr\. [a-z]{7} [a-z]{4}$'
titanic_df['Name'].str.match(pattern, flags=re.I)

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888    False
889    False
890    False
Name: Name, Length: 891, dtype: bool

In [18]:
# Jetzt der Ausschnitt:
titanic_df[titanic_df['Name'].str.match(pattern, flags=re.I)]

,Survived,Name,Sex,Ticket,Last_Name,First_Name
733,0,"Berriman, Mr. William John",male,28425,Berriman,Mr. William John


In [19]:
# Aufgabe:
# matcht alle Personen, die in Klammern zwei Namen haben
# Erster ist fünf Zeichen, zweiter sechs Zeichen lang


In [32]:
# Mit extract kann man Dinge, wie der Name schon sagt, extrahieren,
# also Substrings nach Pattern herausholen. Dazu muss man Gruppen einfangen, wie schon bei re.
# Versuchen wir mal die Mädchennamen zu extrahieren,
# also die Namensteile in Klammern bei den Mrs.
# Erst einmal nehmen wir uns einen Ausschnitt, der nur Mrs. enthält heraus:
mrs_df = titanic_df[titanic_df['Name'].str.contains(r'Mrs\.', regex=True)]
mrs_df.head()

,Survived,Name,Sex,Ticket,Last_Name,First_Name
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,Cumings,Mrs. John Bradley (Florence Briggs Thayer)
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,Futrelle,Mrs. Jacques Heath (Lily May Peel)
8,1,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742,Johnson,Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9,1,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736,Nasser,Mrs. Nicholas (Adele Achem)
15,1,"Hewlett, Mrs. (Mary D Kingcome)",female,248706,Hewlett,Mrs. (Mary D Kingcome)


In [21]:
# Kleiner Test:
mrs_df['Sex'].value_counts()
# Grad nochmal Glück gehabt ;)

Sex
female    125
Name: count, dtype: int64

In [22]:
# Pattern sagt: Öffnende Klammer, dann irgend etwas, was keine Klammer ist, mindestens ein Mal,
# dann schließende Klammer. Um den Teil zwischen den Klammern sind noch einmal Klammern,
# um die Gruppe einzufangen:
pattern = r'\(([^()]+)\)'
mrs_df['Name'].str.extract(pattern)

,0
1,Florence Briggs Thayer
3,Lily May Peel
8,Elisabeth Vilhelmina Berg
9,Adele Achem
15,Mary D Kingcome
...,...
871,Sallie Monypeny
874,Hannah Wizosky
879,Lily Alexenia Wilson
880,Imanita Parrish Hall


In [23]:
# Spaltenbenennung
pattern = r'\((?P<maiden_name>.*)\)'
mrs_df['Name'].str.extract(pattern)

,maiden_name
1,Florence Briggs Thayer
3,Lily May Peel
8,Elisabeth Vilhelmina Berg
9,Adele Achem
15,Mary D Kingcome
...,...
871,Sallie Monypeny
874,Hannah Wizosky
879,Lily Alexenia Wilson
880,Imanita Parrish Hall


In [24]:
# Hier war kein Mädchenname angegeben:
mrs_df.loc[19]

Survived                            1
Name          Masselmani, Mrs. Fatima
Sex                            female
Ticket                           2649
Last_Name                  Masselmani
First_Name                Mrs. Fatima
Name: 19, dtype: object

In [38]:
# Aufgabe:
# Extrahiere bei allen Mrs. den ersten Vornamen mittels extract (und speichere das Ergebnis in mrs_df):
first_name_patt = r'Mrs\. (\w+)'
mrs_df_2 = mrs_df['First_Name'].str.extract(first_name_patt, flags=re.I).dropna()
mrs_df_2

,0
1,John
3,Jacques
8,Oscar
9,Nicholas
18,Julius
...,...
871,Richard
874,Samuel
879,Thomas
880,William


In [39]:
# Was ist mit den Nans?
mrs_df.loc[[15], 'Name']

15    Hewlett, Mrs. (Mary D Kingcome) 
Name: Name, dtype: object

In [40]:
# Slicing: Den ersten Buchstaben vom Nachnamen:
titanic_df['Last_Name'].str.slice(0, 1)

0      B
1      C
2      H
3      F
4      A
      ..
886    M
887    G
888    J
889    B
890    D
Name: Last_Name, Length: 891, dtype: object

In [41]:
# replace ersetzt Strings und kann auch nach Pattern ersetzen.
# Wir wollen (warum auch immer) alle reinen Zahlenketten in der Spalte Ticket schwärzen durch XXX:
titanic_df['Ticket'].str.replace(r'\b(\d+)\b', 
                                 repl='XXX', 
                                 regex=True)

0         A/XXX XXX
1            PC XXX
2      STON/O2. XXX
3               XXX
4               XXX
           ...     
886             XXX
887             XXX
888       W./C. XXX
889             XXX
890             XXX
Name: Ticket, Length: 891, dtype: object

In [109]:
# Aufgabe:
# Du willst unbedingt die Initialien als zusätzliche erstellen. Wie machst du das?
# Was musst du bei der Lösung alles bedenken?

In [55]:
# Extrahiere die ersten Buchstaben aus "First_Name" und "Last_Name"
titanic_df['cleaned_First_Name'] = titanic_df['First_Name'].str.extract(r'(?:Mrs\.|Mr\.)\s[\w\s]+\(([\w]+)')
titanic_df['Initialen'] = titanic_df['cleaned_First_Name'].str[0] + titanic_df['Last_Name'].str[0]
titanic_df['Initialen'].dropna()

1      FC
3      LF
8      EJ
9      AN
18     EV
       ..
871    SB
874    HA
879    LP
880    IS
885    MR
Name: Initialen, Length: 109, dtype: object

In [42]:
# find > gibt Indexposition des erstes Vorkommnisses eines Substrings 
# in Leserichtung (links nach rechts) aus. Wenn nichts gefunden wird, dann ist der Wert -1.
# Am Beispiel von Namen, die 'John' enthalten (Boolean Mask):
titanic_df['Name'].str.find('John')

0      -1
1      14
2      -1
3      -1
4      -1
       ..
886    -1
887    -1
888     0
889    -1
890    -1
Name: Name, Length: 891, dtype: int64

In [43]:
# Mit rfind sucht man von rechts aus den Index.
# Für die Deutlichkeit suchen wir nur nach dem Buchstaben n.
# Es kommen recht hohe Indexpositionen heraus:
titanic_df['Name'].str.rfind('n')

0      15
1      33
2      20
3      -1
4      21
       ..
886     2
887    -1
888    30
889    -1
890    -1
Name: Name, Length: 891, dtype: int64

In [112]:
# Zum Vergleich die Indexpositionen bei der Suche von links aus:
titanic_df['Name'].str.find('n')

0      4
1      4
2      6
3     -1
4      4
      ..
886    2
887   -1
888    3
889   -1
890   -1
Name: Name, Length: 891, dtype: int64

In [113]:
# findall findet alle Vorkommnisse und gibt eine Liste aus, wo diese gefunden wurden:
titanic_df['Name'].str.findall('.*on.*')

0                                              []
1                                              []
2                                              []
3                                              []
4                                              []
                          ...                    
886                       [Montvila, Rev. Juozas]
887                                            []
888    [Johnston, Miss. Catherine Helen "Carrie"]
889                                            []
890                                            []
Name: Name, Length: 891, dtype: object